# Project Data Warehouse Infrastructure Setup

1. Identity and Access Management (IAM) Users Creation and Deletion: to manage access to AWS services and resources.
2. IAM Roles Creation and Deletion: to delegate permissions to AWS services.
3. Security Groups Creation and Deletion: to control inbound and outbound traffic to the AWS resources.
4. Redshift Cluster Creation and Deletion: to store and manage the data warehouse.

In [29]:
# Import the necessary modules and packages
import boto3
import configparser
import json
import sys
import time
import logging
import pandas as pd
from time import sleep
from botocore.exceptions import ClientError
from IPython.display import clear_output

## IAM Users Creation and Deletion
---

In [2]:
"""
This code reads AWS credentials and configuration from the dwh.cfg file. If system_admin_key 
and system_admin_secret are provided in the configuration file, an IAM client is created using 
the 'system_admin' user's AWS credentials. Otherwise, an IAM client is created using the AWS 
credentials generated from the Launch Cloud Gateway.
"""
# Read the AWS credentials and configuration from the dwh.cfg file
config = configparser.ConfigParser()

try:
    with open('dwh.cfg') as config_file:
        config.read_file(config_file)
except FileNotFoundError as e:
    print(f"Error reading configuration file: {str(e)}")
    exit(1)

        
# Get the AWS credentials from the dwh.cfg file    
KEY = config.get('AWS', 'gateway_key')
SECRET = config.get('AWS', 'gateway_secret')
AWS_SESSION_TOKEN= config.get('AWS', 'gateway_aws_session_token')

system_admin_key = config.get('AWS', 'system_admin_key')
system_admin_secret = config.get('AWS', 'system_admin_secret') 

if system_admin_key and system_admin_secret:
    # Create an IAM client using the AWS credentials generated by the 'system_admin' user 
    iam = boto3.client('iam',
                      aws_access_key_id=system_admin_key,
                      aws_secret_access_key=system_admin_secret)

else:
    # Create an IAM client using the AWS credentials generated from the Launch Cloud Gateway 
    iam = boto3.client('iam',
                      aws_access_key_id=KEY,
                      aws_secret_access_key=SECRET,
                      aws_session_token=AWS_SESSION_TOKEN)

In [3]:
"""
The following code creates an IAM user with AdministratorAccess policy and saves the user's 
access key ID and secret to the dwh.cfg configuration file.

Raises:

    -> FileNotFoundError: If the dwh.cfg file is not found.
    -> configparser.NoSectionError: If the required section is missing from the dwh.cfg file.
    -> configparser.NoOptionError: If the required option is missing from the dwh.cfg file.
    -> iam.exceptions.NoSuchEntityException: If the IAM user does not exist.
    -> Exception: If an error occurs while creating the IAM user or attaching policies to the user.
    
The code reads the AWS credentials and configuration from the dwh.cfg file using the 
configparser module. It then retrieves the name of the IAM user from the configuration file. 
If the IAM user already exists, the code prints a message indicating this. Otherwise, 
the code creates the IAM user with the AdministratorAccess policy, attaches the policy to the user, 
and creates access keys for the user. The access key ID and secret are then saved to the dwh.cfg file.

Note that the iam client used in this code is assumed to have already been created earlier in the program.
"""
# Read the AWS credentials and configuration from the dwh.cfg file
config = configparser.ConfigParser()

try:
    with open('dwh.cfg') as config_file:
        config.read_file(config_file)
except FileNotFoundError as e:
    print(f"Error reading configuration file: {str(e)}")
    exit(1)

# Specify the IAM user name    
try:
    admin_username = config.get('AWS', 'admin_username')
except configparser.NoSectionError:
    print("Error: Configuration file is missing required section.")
    exit(1)
except configparser.NoOptionError:
    print("Error: Configuration file is missing required option.")
    exit(1)
    
# Create an IAM user with AdministratorAccess policy
try:
    # Check if the IAM user already exists
    iam.get_user(UserName=admin_username)
    print(f"IAM user {admin_username} already exists")
# If the IAM user does not exist, create it
except iam.exceptions.NoSuchEntityException:
    try:
        response = iam.create_user(UserName=admin_username)
        print(f"IAM user {admin_username} created successfully")
    except Exception as e:
        print(f"Error creating IAM user: {str(e)}")
        exit(1)

    # Attach policies to the IAM user   
    policy_arns = ['arn:aws:iam::aws:policy/AdministratorAccess', 
                   'arn:aws:iam::aws:policy/AmazonRedshiftFullAccess',
                  'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess']
    
    for arn in policy_arns:
        try:
            response = iam.attach_user_policy(UserName=admin_username, PolicyArn=arn)
        except Exception as e:
            print(f"Error attaching IAM policy: {str(e)}")
            exit(1)

    # Create access keys for the IAM user
    try:
        access_key = iam.create_access_key(UserName=admin_username)['AccessKey']
    except Exception as e:
        print(f"Error creating IAM access key: {str(e)}")
        exit(1)

    # Save the new access key ID and secret in your dwh.cfg file
    config.set('AWS', 'system_admin_key', access_key['AccessKeyId'])
    config.set('AWS', 'system_admin_secret', access_key['SecretAccessKey'])

    with open('dwh.cfg', 'w') as config_file:
        config.write(config_file)


IAM user system_admin already exists


In [4]:
"""
To delete a user along with their secret key and policies, set the 'delete_user' flag to True 
and ensure that the 'delete_username' matches the username of the user to be deleted.

This code performs the deletion of an IAM user from AWS, along with their associated 
policies and access key. The user to be deleted is specified using the variable delete_username. 
The access key is retrieved from the dwh.cfg configuration file based on whether the user to be deleted 
is the data_engineer_username or system_admin_username.

If the delete_user flag is set to True, the access key associated with the specified user is deleted first. 
Then, all policies attached to the user are detached, and the user is deleted. If the user to be deleted is 
the system_admin_username, the system_admin_key and system_admin_secret values in the dwh.cfg configuration 
file are set to empty strings. If the user to be deleted is the data_engineer_username, the data_engineer_key 
and data_engineer_secret values in the dwh.cfg configuration file are set to empty strings. Finally, 
the updated dwh.cfg configuration file is written back to disk.

The code includes a try-except block to handle situations where the dwh.cfg file is not found. 
If an exception is raised, the script will print an error message and exit with a status of 1.
"""
# Read the AWS credentials and configuration from the dwh.cfg file
config = configparser.ConfigParser()

try:
    with open('dwh.cfg') as config_file:
        config.read_file(config_file)
except FileNotFoundError as e:
    print(f"Error reading configuration file: {str(e)}")
    exit(1)

delete_username = admin_username

delete_user = False

if delete_user:
    if delete_username == admin_username:
        access_key = config.get('AWS', 'system_admin_key')
        
    # Delete the access key
    try:
        iam.delete_access_key(
            AccessKeyId=access_key,
            UserName=delete_username
        )
    except iam.exceptions.NoSuchEntityException:
        print("Access key does not exist for user:", delete_username)

    # Detach all policies from the user
    attached_policies = iam.list_attached_user_policies(UserName=delete_username)
    for policy in attached_policies['AttachedPolicies']:
        try:
            iam.detach_user_policy(UserName=delete_username, PolicyArn=policy['PolicyArn'])
        except iam.exceptions.NoSuchEntityException:
            print("Policy does not exist for user:", delete_username)

    # Delete the user
    try:
        iam.delete_user(
            UserName=delete_username
        )
    except iam.exceptions.NoSuchEntityException:
        print("User does not exist:", delete_username)
    
    if delete_username == admin_username:
        print(f"User: {delete_username} deleted...")
        # Remove the access key ID and secret in your dwh.cfg file
        config.set('AWS', 'system_admin_key', '')
        config.set('AWS', 'system_admin_secret', '')

    with open('dwh.cfg', 'w') as config_file:
        config.write(config_file)
else:
    print("User deletion skipped...")


User deletion skipped...


## IAM Roles Creation and Deletion
---

In [4]:
# Use the system admin key and secret, which was set up with administrative access
KEY                    = config.get('AWS','system_admin_key')
SECRET                 = config.get('AWS','system_admin_secret')

# Create an IAM client with the provided credentials
iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-east-1'
                  )

In [7]:
"""
This code creates an AWS IAM role with a specified name and attaches an Amazon S3 
policy to it. If the role already exists, the code retrieves its ARN. 
The ARN is then saved to a configuration file.
"""
# Define the IAM role name
role_name = 'myDataWarehouseRedshiftRole'

# Define the policy ARN to attach to the role
policy_arn = 'arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'

# Create the role if it doesn't exist
try:
    role = iam.get_role(RoleName=role_name)
    roleArn = role['Role']['Arn']
    print(f"The IAM Role '{role_name}' already exists.")
    
except iam.exceptions.NoSuchEntityException:
    # Create the role if it doesn't exist
    try:
        print(f"Creating a new IAM Role '{role_name}'")
        dwhRole = iam.create_role(
            Path='/',
            RoleName=role_name,
            Description="Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps({
                'Statement': [{
                    'Action': 'sts:AssumeRole',
                    'Effect': 'Allow',
                    'Principal': {'Service': 'redshift.amazonaws.com'}
                }],
                'Version': '2012-10-17'
            })
        )
         
        # Attach the policy to the role
        iam.attach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
        print(f"The IAM Role '{role_name}' was created successfully")
        
    except Exception as e:
        print(f"An error occurred while creating the IAM Role: {e}")
        raise e

# Get the IAM role ARN
roleArn = iam.get_role(RoleName=role_name)['Role']['Arn']

print("Arn: ", roleArn)
# Save the new IAM role ARN in the dwh.cfg file
config.set('IAM_ROLE', 'arn', roleArn)

with open('dwh.cfg', 'w') as config_file:
    config.write(config_file)



The IAM Role 'myDataWarehouseRedshiftRole' already exists.
Arn:  arn:aws:iam::065061674598:role/myDataWarehouseRedshiftRole


In [8]:
"""
To delete the I AM role with policies, set the 'delete_role' flag to True 
and make sure the 'delete_role_name' is correct.

Delete an IAM role with a specified name. If the delete_role flag is set to True, 
detach all policies from the role and delete the role. If the role doesn't exist, 
notify the user.
"""

delete_role = False

if delete_role:
    # Define the name of the role to delete
    delete_role_name = 'myDataWarehouseRedshiftRole'

    try:
        # Check if the role exists
        role = iam.get_role(RoleName=delete_role_name)

        # Detach all policies from the role
        response = iam.list_attached_role_policies(RoleName=delete_role_name)
        for policy in response['AttachedPolicies']:
            iam.detach_role_policy(RoleName=delete_role_name, PolicyArn=policy['PolicyArn'])

        # Delete the role
        iam.delete_role(RoleName=delete_role_name)

        print(f"The IAM Role '{delete_role_name}' was successfully deleted.")
        
    except iam.exceptions.NoSuchEntityException:
        print(f"The IAM Role '{delete_role_name}' does not exist.")
    
    except Exception as e:
        print(f"An error occurred while deleting the IAM Role: {e}")
        raise e
else:
    print("IAM role deletion skipped...")

IAM role deletion skipped...


## Security Groups Creation and Deletion
---

In [9]:
"""
This code creates a security group in AWS EC2 for a Redshift cluster with TCP port 5439 open 
to all IP addresses. It first checks if the security group exists, and if not, it creates a 
new one and adds the required inbound rule. The system admin key and secret are used 
for authentication.
"""

# Use the system admin key and secret, which was set up with administrative access
KEY                    = config.get('AWS','system_admin_key')
SECRET                 = config.get('AWS','system_admin_secret')

# Create an ec2 client with the provided credentials
ec2 = boto3.client('ec2',
                   region_name="us-east-1",
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

security_group_name = 'redshift_security_group'

create_security_group = True

if create_security_group:
    try:
        # Check if security group with the given name already exists
        response = ec2.describe_security_groups(GroupNames=[security_group_name])
        
        # If the security group exists, don't create a new one
        if response['SecurityGroups']:
            print(f"Security group '{security_group_name}' already exists. Skipping creation.")
            security_group_id = response['SecurityGroups'][0]['GroupId']
  
    # If the security group doesn't exist, create a new one
    except ClientError as e:
        # Security group does not exist, create it and authorize ingress
        if e.response['Error']['Code'] == 'InvalidGroup.NotFound':
            # Filter the default VPC in the EC2 service and retrieve its ID
            response = ec2.describe_vpcs(Filters=[{'Name': 'isDefault','Values': ['true']}])
            vpc_id = response['Vpcs'][0]['VpcId']

            # Create a security group
            security_group = ec2.create_security_group(
                GroupName=security_group_name,
                Description='Authorise redshift cluster access',
                VpcId=vpc_id
            )

            # Get the security group ID
            security_group_id = security_group['GroupId']

            # Authorize ingress on the security group
            response = ec2.authorize_security_group_ingress(
                GroupId=security_group_id,
                IpPermissions=[
                    {
                        'IpProtocol': 'tcp',
                        'FromPort': 5439,
                        'ToPort': 5439,
                        'IpRanges': [
                            {'CidrIp': '0.0.0.0/0'}
                        ]
                    }
                ]
            )
            print(f"Security group '{security_group_name}' created with ID {security_group_id}.")
        else:
            # Some other error occurred
            print("Unexpected error:", e)
else:
    print("Security group creation skipped...")

Security group 'redshift_security_group' already exists. Skipping creation.


In [10]:
"""
This code checks if a security group exists and deletes it if it does. If the security group 
doesn't exist, it prints a message. The deletion is controlled by a boolean flag.
"""

delete_security_group_name = 'redshift_security_group'

delete_security = False

if delete_security:
    try:
        # Check if security group with the given name already exists
        response = ec2.describe_security_groups(GroupNames=[delete_security_group_name])

        # If the security group exists, delete it
        if response['SecurityGroups']:
            security_group_id = response['SecurityGroups'][0]['GroupId']
            ec2.delete_security_group(GroupId=security_group_id)
            print(f"Security group '{delete_security_group_name}' with ID {security_group_id} deleted.")

    # If the security group doesn't exist, don't do anything
    except ClientError as e:
        if e.response['Error']['Code'] == 'InvalidGroup.NotFound':
            print(f"Security group '{delete_security_group_name}' does not exist. Skipping deletion.")
        else:
            # Some other error occurred
            print("Unexpected error:", e)

else:
    print("Security group deletion skipped...")


Security group deletion skipped...


## Redshift Clusters Creation and Deletion
---

In [19]:
"""
Read configuration values from a configuration file 'dwh.cfg' and create a Redshift 
client using Boto3 library.

Attributes:

DWH_CLUSTER_TYPE: string value for the type of Redshift cluster.
DWH_NODE_TYPE: string value for the node type of Redshift cluster.
DWH_NUM_NODES: integer value for the number of nodes in the Redshift cluster.
DWH_DB: string value for the name of the Redshift database.
DWH_CLUSTER_IDENTIFIER: string value for the identifier of the Redshift cluster.
DWH_DB_USER: string value for the user name to connect to the Redshift cluster.
DWH_DB_PASSWORD: string value for the password to connect to the Redshift cluster.
DWH_PORT: integer value for the port number of the Redshift cluster.
DWH_SUBNET_GROUP_NAME: string value for the name of the subnet group of the Redshift cluster.
DWH_REDSHIFT_SECURITY_GROUP: string value for the name of the security group of the Redshift cluster.
roleArn: string value for the Amazon Resource Name (ARN) of the IAM role.
KEY: string value for the system administrator key for AWS.
SECRET: string value for the system administrator secret for AWS.

"""

try:
    with open('dwh.cfg') as config_file:
        config.read_file(config_file)
except FileNotFoundError as e:
    print(f"Error reading configuration file: {str(e)}")
    exit(1)
    
# Get the configuration values
try:
    DWH_CLUSTER_TYPE                 = config.get('DWH','dwh_cluster_type')
    DWH_NODE_TYPE                    = config.get('DWH','dwh_node_type')
    DWH_NUM_NODES                    = int(config.get('DWH','dwh_num_nodes'))
    DWH_DB                           = config.get('DWH','dwh_db')
    DWH_CLUSTER_IDENTIFIER           = config.get('DWH','dwh_cluster_identifier')
    DWH_DB_USER                      = config.get('DWH','dwh_db_user')
    DWH_DB_PASSWORD                  = config.get('DWH','dwh_db_password')
    DWH_PORT                         = int(config.get('DWH','dwh_port'))
    DWH_SUBNET_GROUP_NAME            = config.get('DWH','dwh_subnet_group_name')
    DWH_REDSHIFT_SECURITY_GROUP      = config.get('DWH','dwh_redshift_security_group_name')
    
    roleArn                 = config.get('IAM_ROLE','arn')
    
    KEY                     = config.get('AWS','system_admin_key')
    SECRET                  = config.get('AWS','system_admin_secret')
    
except KeyError as e:
    print(f"Missing configuration value: {e}")
    exit(1)


# Create a Redshift client
try:
    redshift = boto3.client('redshift',
                           region_name="us-east-1",
                           aws_access_key_id=KEY,
                           aws_secret_access_key=SECRET
                           )
except Exception as e:
    print(f"Failed to create Redshift client: {e}")
    exit(1)


In [20]:
print(DWH_CLUSTER_IDENTIFIER)

dataWarehouseRedshiftCluster


In [21]:
"""
This code retrieves security groups with the specified name of 'redshift_security_group' and extracts
their IDs from the response. 
"""

try:
    # retrieve the security groups with the specified name of 'redshift_security_group'
    response = ec2.describe_security_groups(Filters=[{'Name': 'group-name','Values': [DWH_REDSHIFT_SECURITY_GROUP]}])

    if 'SecurityGroups' in response:
        # extract the security group IDs from the response
        security_group_ids = [sg['GroupId'] for sg in response['SecurityGroups']]
        print(f"Found {len(security_group_ids)} security group(s) with name {DWH_REDSHIFT_SECURITY_GROUP}")
    else:
        print(f"No security groups found with name {DWH_REDSHIFT_SECURITY_GROUP}")
        security_group_ids = []
except Exception as e:
    print(f"Error retrieving security groups: {e}")
    security_group_ids = []

Found 1 security group(s) with name redshift_security_group


In [22]:
""" 
This script provides functions for creating and managing Amazon Redshift clusters. It includes a function
to check if a cluster exists, create a new cluster, and wait for it to become available. It requires the 
'boto3' and 'logging' modules to be installed and the cluster configuration to be specified 
in the calling script. If the 'should_create_cluster' flag is True, the script creates the cluster.

"""

# Configure logging
logging.basicConfig(level=logging.INFO)

# Set the flag to control whether to create the cluster or not
should_create_cluster = True

# Function to check if a Redshift cluster exists
def check_cluster_exists(redshift, cluster_identifier):
    try:
        response = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)
        cluster = response['Clusters'][0]
        return True if cluster['ClusterIdentifier'] == cluster_identifier else False
    except redshift.exceptions.ClusterNotFoundFault:
        return False
    except Exception as e:
        logging.error(f"Error checking Redshift cluster {cluster_identifier}: {e}")
        exit(1)

# Function to create a new Redshift cluster
def create_cluster(redshift, cluster_config, role_arn, security_group_ids):
    try:
        response = redshift.create_cluster(
            ClusterType=cluster_config['type'],
            NodeType=cluster_config['node_type'],
            NumberOfNodes=cluster_config['num_nodes'],
            DBName=cluster_config['db_name'],
            ClusterIdentifier=cluster_config['identifier'],
            MasterUsername=cluster_config['user'],
            MasterUserPassword=cluster_config['password'],
            IamRoles=[role_arn],
            VpcSecurityGroupIds=security_group_ids,
            PubliclyAccessible=True,
            Port=cluster_config['port']
        )
        logging.info(f"Redshift cluster '{cluster_config['identifier']}' is being created...")
        return response
    except Exception as e:
        logging.error(f"Error creating Redshift cluster '{cluster_config['identifier']}': {e}")
        exit(1)

# Function to wait until the cluster becomes available
def wait_cluster_available(redshift, cluster_identifier):
    while True:
        time.sleep(10)  # Wait for 10 seconds before checking the cluster status again
        response = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)
        cluster = response['Clusters'][0]
        if cluster['ClusterStatus'] == 'available':
            break
        logging.info(f"Waiting for Redshift cluster '{cluster_identifier}' to become available...")

# Main script
if should_create_cluster:
    if not check_cluster_exists(redshift, DWH_CLUSTER_IDENTIFIER):
        cluster_config = {
            'type': DWH_CLUSTER_TYPE,
            'node_type': DWH_NODE_TYPE,
            'num_nodes': DWH_NUM_NODES,
            'db_name': DWH_DB,
            'identifier': DWH_CLUSTER_IDENTIFIER,
            'user': DWH_DB_USER,
            'password': DWH_DB_PASSWORD,
            'port': DWH_PORT
        }
        create_cluster(redshift, cluster_config, roleArn, security_group_ids)
        wait_cluster_available(redshift, DWH_CLUSTER_IDENTIFIER)
        logging.info(f"Redshift cluster '{DWH_CLUSTER_IDENTIFIER}' has been successfully created and is available.")
    else:
        logging.info(f"Redshift cluster '{DWH_CLUSTER_IDENTIFIER}' already exists. Skipping creation.")
else:
    logging.info("Redshift cluster creation skipped.")


INFO:root:Redshift cluster 'dataWarehouseRedshiftCluster' is being created...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become available...
INFO:root:Waiting for Redshift cluster 'dataWarehouseRedshiftCluster' to become avail

In [24]:
"""
This code reads the AWS Redshift cluster details from a configuration file, updates 
the host address with the cluster's endpoint, and writes the updated configuration back to the file. 
The endpoint address is also printed for verification.

"""

# Load config file
config = configparser.ConfigParser()
config.read('dwh.cfg')

# Get cluster properties
cluster_props = redshift.describe_clusters(ClusterIdentifier=config.get('DWH','DWH_CLUSTER_IDENTIFIER'))['Clusters'][0]

# Get cluster endpoint
host = cluster_props['Endpoint']['Address']

# Update config file with host
config.set('CLUSTER', 'host', host)

# Write changes to config file
with open('dwh.cfg', 'w') as configfile:
    config.write(configfile)

# Print host for verification
print("Host: ", host)


Host:  datawarehouseredshiftcluster.c8mixlfqn1ra.us-east-1.redshift.amazonaws.com


In [27]:
"""
This script deletes an Amazon Redshift cluster. It requires the 'boto3' module to be installed 
and the cluster identifier to be specified in the 'CLUSTER_IDENTIFIER' constant. If 'delete_cluster' 
is True, the script checks if the cluster exists, deletes it, and waits for it to be deleted. 
If the cluster does not exist, the script prints a message and exits. If 'delete_cluster' is False, 
the script prints a message and skips the deletion.

"""
# Set the cluster identifier for the cluster to be deleted
CLUSTER_IDENTIFIER = DWH_CLUSTER_IDENTIFIER

delete_cluster = False

if delete_cluster:
    # Check if the cluster exists
    try:
        response = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)
        cluster = response['Clusters'][0]
        if cluster['ClusterIdentifier'] == CLUSTER_IDENTIFIER:
            # Try to delete the cluster
            try:
                response = redshift.delete_cluster(
                    ClusterIdentifier=CLUSTER_IDENTIFIER,
                    SkipFinalClusterSnapshot=True
                )
                print(f"Redshift cluster {CLUSTER_IDENTIFIER} is being deleted...")
            except Exception as e:
                print(f"Error deleting redshift cluster {CLUSTER_IDENTIFIER}")
                exit(1)

            # Wait for the cluster to be deleted
            print("Waiting for the cluster to be deleted...")
            while True:
                try:
                    response = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)
                    cluster = response['Clusters'][0]
                    if cluster['ClusterStatus'] == 'deleting':
                        # Display the waiting animation while the cluster is being deleted
                        for i in range(10):
                            clear_output(wait=True)
                            print("Waiting for the cluster to be deleted" + "." * i)
                            sleep(0.1)
                    else:
                        print(f"Redshift cluster {CLUSTER_IDENTIFIER} has been successfully deleted.")
                        break
                except redshift.exceptions.ClusterNotFoundFault:
                    print(f"Redshift cluster {CLUSTER_IDENTIFIER} has been successfully deleted.")
                    break
                except Exception as e:
                    print(f"Error checking redshift cluster {CLUSTER_IDENTIFIER}: {e}")
                    exit(1)
    except redshift.exceptions.ClusterNotFoundFault:
        print(f"Redshift cluster {CLUSTER_IDENTIFIER} not found.")
        exit(1)
    except Exception as e:
        print(f"Error checking redshift cluster {CLUSTER_IDENTIFIER}: {e}")
        exit(1)
else:
    print("Redshift cluster deletion skipped...")

Redshift cluster deletion skipped...


In [1]:
%run create_tables.py

Running the create_tables.py file...
DROP TABLE IF EXISTS staging_events;
DROP TABLE IF EXISTS staging_songs;
DROP TABLE IF EXISTS songplays;
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS songs;
DROP TABLE IF EXISTS artists;
DROP TABLE IF EXISTS time;

CREATE TABLE IF NOT EXISTS staging_events (
    artist          VARCHAR,
    auth            VARCHAR,
    firstName       VARCHAR,
    gender          VARCHAR,
    itemInSession   INTEGER,
    lastName        VARCHAR,
    length          FLOAT,
    level           VARCHAR,
    location        VARCHAR,
    method          VARCHAR,
    page            VARCHAR,
    registration    BIGINT,
    sessionId       INTEGER,
    song            VARCHAR,
    status          INTEGER,
    ts              BIGINT,
    userAgent       VARCHAR,
    userId          INTEGER
);


CREATE TABLE IF NOT EXISTS staging_songs (
    song_id         VARCHAR,
    num_songs       INTEGER,
    title           VARCHAR,
    artist_name     VARCHAR,
    artist_latitude

In [2]:
%run etl.py

Running the etl.py file...

COPY staging_events 
FROM 's3://udacity-dend/log_data'
CREDENTIALS 'aws_iam_role=arn:aws:iam::065061674598:role/myDataWarehouseRedshiftRole'
REGION 'us-west-2'
FORMAT AS JSON 's3://udacity-dend/log_json_path.json';


COPY staging_songs 
FROM 's3://udacity-dend/song_data'
CREDENTIALS 'aws_iam_role=arn:aws:iam::065061674598:role/myDataWarehouseRedshiftRole'
REGION 'us-west-2'
FORMAT AS JSON 'auto';


INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT
    TIMESTAMP 'epoch' + (e.ts / 1000) * INTERVAL '1 second' AS start_time,
    e.userId AS user_id,
    e.level,
    s.song_id,
    s.artist_id,
    e.sessionId AS session_id,
    e.location,
    e.userAgent AS user_agent
FROM staging_events e
JOIN staging_songs s ON e.song = s.title AND e.artist = s.artist_name AND e.length = s.duration
WHERE e.page = 'NextSong' AND e.userId IS NOT NULL;


INSERT INTO users (user_id, first_name, last_name, gender, level

NameError: name 'cut' is not defined

In [4]:
%run etl.py

Running the etl.py file...
The number of records in each table: 

    SELECT COUNT(*) FROM staging_events

(8056,)



    SELECT COUNT(*) FROM staging_songs

(14896,)



    SELECT COUNT(*) FROM songplays

(319,)



    SELECT COUNT(*) FROM users

(104,)



    SELECT COUNT(*) FROM songs

(14896,)



    SELECT COUNT(*) FROM artists

(10025,)



    SELECT COUNT(*) FROM time

(319,)



SELECT s.title, a.name, COUNT(*) as play_count
FROM songplays sp
JOIN songs s ON sp.song_id = s.song_id
JOIN artists a ON sp.artist_id = a.artist_id
GROUP BY s.title, a.name
ORDER BY play_count DESC
LIMIT 10;

most_played_songs_query selects the top 10 most played songs:

(song title, artists name, play count): 

("You're The One", 'Dwight Yoakam', 37)
('Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Lonnie Gordon', 9)
("I CAN'T GET STARTED", 'Ron Carter', 9)
("Nothin' On You [feat. Bruno Mars] (Album Version)", 'B.o.B', 8)
("Hey Daddy (Daddy's Home)", 'Usher featuring Jermaine Dupri', 6)
("Hey 

In [3]:
print("Completed Project 2: Data Warehouse")

Completed Project 2: Data Warehouse
